## Ejercicio 2: Leer un archivo de texto y generar resúmenes en tres tonos diferentes

En este ejercicio, usaremos un fichero .txt con un reporte financiero para resumirlo, adaptando el tono a tres audiencias diferentes

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

print(type(OPENAI_API_KEY),OPENAI_API_KEY)
print(type(API_VERSION),API_VERSION)
print(type(AZURE_ENDPOINT),AZURE_ENDPOINT)

In [2]:
from openai import AzureOpenAI

##### Establecemos el valor de la API key de OpenAI. En una aplicación real, no podemos exponer esta API key en el código, y la cogeríamos con una variable de entorno mediante un os.getenv()

In [4]:
client = AzureOpenAI(
            api_key = OPENAI_API_KEY,
            api_version=API_VERSION,
            azure_endpoint= AZURE_ENDPOINT# Este endpoint corresponde a la región france-central
)

try:
    with open("financial_report.txt", "r", encoding="utf-8") as f:
        contenido = f.read()
except FileNotFoundError:
    print("El archivo 'financial_report.txt' no se encontró. Por favor, crea el archivo y vuelve a ejecutar la celda.")
    contenido = ""

def resumir_texto(prompt_template: str, texto: str) -> str:
    """Rellena el template del prompt con el texto y devuelve el resumen generado."""
    prompt = prompt_template.replace("<<TEXTO>>", texto)
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=2000,
    )
    return completion.choices[0].message.content.strip()


##### Definimos los tres prompts para los diferentes tonos:

In [5]:
prompt_bullet = (
    "Eres un experto en análisis de textos y resúmenes. A partir del siguiente documento, "
    "debes generar un resumen estructurado en formato de bullet points. "
    "Sigue estas instrucciones detalladas:\n"
    "1. Identifica los puntos clave del documento, tales como datos financieros relevantes, conclusiones principales y recomendaciones importantes.\n"
    "2. Cada bullet point debe contener una única idea clara y estar redactado de forma breve y precisa.\n"
    "3. No incluyas explicaciones o comentarios adicionales; solo presenta la lista de ideas clave.\n"
    "4. Asegúrate de que la información se presente en un orden lógico y coherente.\n"
    "5. Escribe un máximo de 6 bullet points\n\n"
    "Documento:\n<<TEXTO>>\n\n"
    "Lista de bullet points:"
)

prompt_publico = (
    "Eres un comunicador profesional especializado en traducir información técnica a un lenguaje accesible para el público general. "
    "Tu tarea es leer el siguiente documento y generar un resumen que cumpla con estas especificaciones:\n"
    "1. Explica de forma sencilla los puntos más importantes, evitando el uso de jerga técnica o complicada.\n"
    "2. Utiliza un lenguaje coloquial y ejemplos cotidianos para ilustrar conceptos que puedan resultar complejos.\n"
    "3. El resumen debe ser breve, pero lo suficientemente informativo para que cualquier persona sin conocimientos financieros entienda la esencia del documento.\n"
    "4. Organiza la información de forma clara, resaltando los aspectos que impactan al lector promedio.\n\n"
    "Documento:\n<<TEXTO>>\n\n"
    "Resumen para público no especialista:"
)

prompt_experto = (
     "Eres un analista financiero con amplia experiencia en reportes y análisis de datos. "
    "Lee el siguiente documento y elabora un resumen detallado cumpliendo estas directrices:\n"
    "1. Enfócate en los aspectos técnicos y financieros más relevantes del documento.\n"
    "2. Incluye una evaluación de las tendencias financieras, análisis de variaciones en ingresos, gastos y balances, y destaca cualquier indicador clave.\n"
    "3. Utiliza terminología financiera precisa y un lenguaje formal, evitando simplificaciones excesivas.\n"
    "4. Proporciona detalles analíticos que sean útiles para la toma de decisiones en un contexto profesional.\n\n"
    "Documento:\n<<TEXTO>>\n\n"
    "Resumen para un experto financiero:"
)

##### Generamos los tres resúmenes

In [ ]:
from IPython.display import Markdown

if contenido:
    resumen_bullet = resumir_texto(prompt_bullet, contenido)
    print("Resumen en bullet points:")
    display(Markdown(resumen_bullet))
    #print(resumen_bullet, "\n")

In [ ]:
if contenido:
    resumen_publico = resumir_texto(prompt_publico, contenido)
    print("\nResumen para público no especialista:")
    #print(resumen_publico, "\n")
    display(Markdown(resumen_publico))

In [ ]:
if contenido:
    resumen_experto = resumir_texto(prompt_experto, contenido)
    print("\nResumen para un experto financiero:")
    display(Markdown(resumen_experto))
    #print(resumen_experto, "\n")

##### - ¿Qué diferencias encuentras en el nivel de detalle y terminología entre los tres resúmenes?
##### - ¿Cómo afecta el parámetro 'temperature' al estilo del resumen?